## Data Preparation

In [1]:
import pandas as pd

In [4]:
movies_data_path = '../data/raw/movies.csv'
finantial_data_path = '../data/raw/finantials.csv'
opening_data_path = '../data/raw/opening_gross.csv'

In [5]:
fin_data = pd.read_csv(finantial_data_path)
movie_data = pd.read_csv(movies_data_path)
opening_data = pd.read_csv(opening_data_path)

In [6]:
numeric_columns_mask = (movie_data.dtypes == float) | (movie_data.dtypes == int)
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]
movie_data = movie_data[numeric_columns+['movie_title']]

In [7]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']]

In [8]:
fin_movie_data = pd.merge(fin_data, movie_data, on= 'movie_title', how='left')

In [9]:
full_movie_data = pd.merge( opening_data,fin_movie_data, on = 'movie_title', how='left')

In [10]:
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 11)

In [11]:
full_movie_data = full_movie_data.drop(['movie_title','gross'],axis=1)

In [12]:
full_movie_data.columns

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'budget', 'imdb_score'],
      dtype='object')

In [13]:
full_movie_data

,opening_gross,screens,production_budget,worldwide_gross,title_year,aspect_ratio,duration,budget,imdb_score
0,2451.0,10.0,12000000,14616,2015.0,1.85,111.0,12000000.0,7.5
1,8330681.0,2271.0,13000000,60414025,1999.0,1.85,97.0,16000000.0,7.2
2,19883351.0,2704.0,85000000,66941559,2000.0,1.85,100.0,85000000.0,4.8
3,5329240.0,2331.0,20000000,17306648,2009.0,2.35,108.0,22000000.0,5.6
4,923715.0,19.0,20000000,181025343,2013.0,2.35,134.0,20000000.0,8.1
...,...,...,...,...,...,...,...,...,...
2299,24733155.0,3036.0,23600000,102236596,2009.0,2.35,88.0,23600000.0,7.7
2300,20065617.0,3482.0,80000000,170805525,2011.0,2.35,102.0,80000000.0,5.2
2301,15650000.0,3394.0,50000000,55348693,2016.0,2.35,102.0,50000000.0,4.8
2302,4510408.0,NaN,35000000,12506188,2006.0,1.85,83.0,35000000.0,4.2


## Modeling

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import numpy as np

In [15]:
X = full_movie_data.drop(['worldwide_gross'], axis = 1)
y = full_movie_data['worldwide_gross']

In [16]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [17]:
results = cross_validate(pipeline ,X,y,return_train_score=True,cv=10)
results

{'fit_time': array([0.17758083, 0.18194914, 0.17583084, 0.17087436, 0.17590642,
        0.18508387, 0.17644072, 0.17544723, 0.16899371, 0.17597127]),
 'score_time': array([0.        , 0.        , 0.00214338, 0.        , 0.00808001,
        0.        , 0.        , 0.        , 0.00793266, 0.        ]),
 'test_score': array([0.66163763, 0.84856496, 0.64131203, 0.7819192 , 0.77416836,
        0.86230749, 0.76419912, 0.85306376, 0.67552274, 0.6524874 ]),
 'train_score': array([0.91105346, 0.9140962 , 0.9183163 , 0.91613755, 0.91832576,
        0.91492969, 0.91866643, 0.91378583, 0.9201035 , 0.91384172])}

In [18]:
train_score = np.mean(results['train_score'])
test_score = np.mean(results['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9159256441060745
Test Score: 0.7515182690501196


## Hyperparameter tunning

In [19]:
from sklearn.model_selection import GridSearchCV

In [33]:
param_tunning = {'core_model__n_estimators': range(20,501,20)} 

In [34]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [35]:
grid_search= GridSearchCV(estimator,
                       param_grid = param_tunning,
                       scoring='r2',
                       cv=5) 

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35,random_state= 42)

In [37]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('core_model',
                                        GradientBoostingRegressor())]),
             param_grid={'core_model__n_estimators': range(20, 501, 20)},
             scoring='r2')

In [38]:
grid_search.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor(n_estimators=200))])

In [39]:
final_result = cross_validate(grid_search.best_estimator_,X_train,y_train,return_train_score=True,cv=7)

In [40]:
train_score = np.mean(final_result['train_score'])
test_score = np.mean(final_result['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9627561411785589
Test Score: 0.7682382307783928


In [41]:
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('imputer', SimpleImputer()),
  ('core_model', GradientBoostingRegressor(n_estimators=200))],
 'verbose': False,
 'imputer': SimpleImputer(),
 'core_model': GradientBoostingRegressor(n_estimators=200),
 'imputer__add_indicator': False,
 'imputer__copy': True,
 'imputer__fill_value': None,
 'imputer__missing_values': nan,
 'imputer__strategy': 'mean',
 'imputer__verbose': 0,
 'core_model__alpha': 0.9,
 'core_model__ccp_alpha': 0.0,
 'core_model__criterion': 'friedman_mse',
 'core_model__init': None,
 'core_model__learning_rate': 0.1,
 'core_model__loss': 'squared_error',
 'core_model__max_depth': 3,
 'core_model__max_features': None,
 'core_model__max_leaf_nodes': None,
 'core_model__min_impurity_decrease': 0.0,
 'core_model__min_samples_leaf': 1,
 'core_model__min_samples_split': 2,
 'core_model__min_weight_fraction_leaf': 0.0,
 'core_model__n_estimators': 200,
 'core_model__n_iter_no_change': None,
 'core_model__random_state': None,
 'core_model__subsample'

In [26]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor(n_estimators=220,
                                             alpha=0.9,
                                             ccp_alpha=0.0,
                                             criterion='friedman_mse',
                                             init=None,
                                             learning_rate=0.1,
                                             loss='squared_error',
                                             max_depth=3,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_iter_no_change=None,
                                             random_state=None,
                                             subsample=1.0,
                                             tol=0.0001,
                                             validation_fraction=0.1,
                                             verbose=0,
                                             warm_start=False))
])

In [42]:
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor())])

In [43]:
estimator.score(X_test, y_test)

0.7465740546477999

## Saving model

In [29]:
from joblib import dump

In [30]:
dump(estimator, '../models/model.pkl')

['../model/model.pkl']

In [31]:
X_train.columns

Index(['opening_gross', 'screens', 'production_budget', 'title_year',
       'aspect_ratio', 'duration', 'cast_total_facebook_likes', 'budget',
       'imdb_score'],
      dtype='object')